In [1]:
#Realizamos imports de librerias
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import seaborn as sn
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn.utils import shuffle
import numpy as np

##### Descarga de los datos

In [ ]:
!wget https://raw.githubusercontent.com/andres-calvo/modelos-1/master/train.csv -O train.csv
!wget https://raw.githubusercontent.com/andres-calvo/modelos-1/master/test.csv -O test.csv

Carga y limpieza de datos

In [7]:
train=pd.read_csv('train.csv')
train.drop('policy_id', axis=1, inplace=True)
test=pd.read_csv('test.csv')
test.drop('policy_id', axis=1, inplace=True)


In [9]:
#Transformamos los datos de las variables categóricas a una representación numérica
for columns in train.columns:
    if dict(train.dtypes)[columns] == 'object':
        label_encoder = preprocessing.LabelEncoder()
        train[columns] = label_encoder.fit_transform(train[columns])
        test[columns] = label_encoder.fit_transform(test[columns])

# Determinar cuántos valores nulos quieres en cada columna
num_rows = train.shape[0]
num_nulls = int(0.05 * num_rows)  # 5% de los valores nulos

# Crear una lista aleatoria de índices de filas
null_rows = np.random.choice(num_rows, size=num_nulls, replace=False)
# Columnas donde simularemos los nulos
null_columns=['policy_tenure','area_cluster','age_of_car']
# Asignar valores nulos a las celdas correspondientes
train.loc[null_rows, null_columns] = np.nan

train.head(10)

,policy_tenure,age_of_car,age_of_policyholder,area_cluster,population_density,make,segment,model,fuel_type,max_torque,...,is_power_door_locks,is_central_locking,is_power_steering,is_driver_seat_height_adjustable,is_day_night_rear_view_mirror,is_ecw,is_speed_alert,ncap_rating,is_claim,policy_ternure
0,0.515874,0.05,0.644231,0.0,4990,1,0,0,0,5,...,0,0,1,0,0,0,1,0,0,NaN
1,0.672619,0.02,0.375000,11.0,27003,1,0,0,0,5,...,0,0,1,0,0,0,1,0,0,NaN
2,0.841110,NaN,0.384615,NaN,4076,1,0,0,0,5,...,0,0,1,0,0,0,1,0,0,NaN
3,0.900277,0.11,0.432692,16.0,21622,1,3,3,2,0,...,1,1,1,1,1,1,1,2,0,NaN
4,0.596403,0.11,0.634615,17.0,34738,2,0,4,2,8,...,1,1,1,0,1,1,1,2,0,NaN
5,1.018709,0.07,0.519231,18.0,13051,3,4,5,1,4,...,1,1,1,1,0,1,1,3,0,NaN
6,0.097992,0.16,0.403846,19.0,6112,4,2,6,1,3,...,1,1,1,0,0,1,1,5,0,NaN
7,0.509085,0.14,0.423077,20.0,8794,1,2,7,2,0,...,1,1,1,1,1,1,1,2,0,NaN
8,0.282394,0.07,0.298077,19.0,6112,3,4,5,1,4,...,1,1,1,1,0,1,1,3,0,NaN
9,0.566255,0.04,0.442308,21.0,17804,1,2,8,2,0,...,1,1,1,1,1,1,1,0,0,NaN


In [10]:
#Imprimimos la cantidad de NaN
num_nans = train.isna().sum()
print(num_nans)

policy_tenure                           0
age_of_car                           2929
age_of_policyholder                     0
area_cluster                         2929
population_density                      0
make                                    0
segment                                 0
model                                   0
fuel_type                               0
max_torque                              0
max_power                               0
engine_type                             0
airbags                                 0
is_esc                                  0
is_adjustable_steering                  0
is_tpms                                 0
is_parking_sensors                      0
is_parking_camera                       0
rear_brakes_type                        0
displacement                            0
cylinder                                0
transmission_type                       0
gear_box                                0
steering_type                     

In [16]:
test.head(10)

,policy_tenure,age_of_car,age_of_policyholder,area_cluster,population_density,make,segment,model,fuel_type,max_torque,...,is_rear_window_defogger,is_brake_assist,is_power_door_locks,is_central_locking,is_power_steering,is_driver_seat_height_adjustable,is_day_night_rear_view_mirror,is_ecw,is_speed_alert,ncap_rating
0,0.341732,0.00,0.586538,15,4076,1,0,0,0,5,...,0,0,0,0,1,0,0,0,1,0
1,0.307241,0.13,0.442308,20,8794,1,2,7,2,0,...,0,1,1,1,1,1,1,1,1,2
2,0.327924,0.12,0.451923,20,8794,2,0,4,2,8,...,0,0,1,1,1,0,1,1,1,2
3,0.782654,0.01,0.461538,17,34738,1,0,0,0,5,...,0,0,0,0,1,0,0,0,1,0
4,1.233404,0.02,0.634615,17,34738,1,0,0,0,5,...,0,0,0,0,1,0,0,0,1,0
5,0.148635,0.15,0.442308,20,8794,1,2,8,2,0,...,1,1,1,1,1,1,1,1,1,0
6,1.114182,0.09,0.355769,7,16206,1,1,9,0,6,...,0,0,1,1,1,0,0,1,1,2
7,1.113638,0.01,0.471154,5,7788,1,2,7,2,0,...,0,1,1,1,1,1,1,1,1,2
8,0.360385,0.15,0.519231,2,6108,1,1,9,0,6,...,0,0,1,1,1,0,0,1,1,2
9,1.209922,0.06,0.451923,2,6108,1,2,8,2,0,...,1,1,1,1,1,1,1,1,1,0


In [21]:

#Desarrollo modelo regresión logística

#Inicialización.
modelLR = LogisticRegression()

train_data = shuffle(train)
#División de datos en conjunto de evaluación y conjunto de entrenamiento
dataTrain = train_data.drop(['is_claim'], axis=1)
targetTrain = train_data['is_claim']


dataTest = test.drop(['is_claim'], axis=1)
targetTest = test['is_claim']


modelLR.fit(dataTrain, targetTrain)

LogisticRegression()

In [26]:
predictions = modelLR.predict(dataTest)
predictions

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

In [29]:
train_accuracy= accuracy_score(targetTest,predictions)

print('Accuracy')
print( 100*train_accuracy)


Accuracy
93.38377517351233
